<a href="https://colab.research.google.com/github/ZEUS77-dev/LogAnomalyDetection/blob/main/LogAnomalyDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
processed = pd.read_csv('/content/drive/MyDrive/Copy of Processed_data.csv')

In [ ]:
df = processed.sample(600000)

In [ ]:
import gc

In [ ]:
del processed
gc.collect()

0

In [ ]:
df['Status'] = df.Status.map({'normal':1,'abnormal':0})

In [ ]:
corpus = df['Keys'].values.tolist()

### WordPiece tokeniser

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer

tokenizer = Tokenizer(models.WordPiece(unl_token="[UNK]"))

In [ ]:
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

In [ ]:
batch_size= 5000

In [ ]:
def batch_iterator():
    for i in range(0,len(corpus),batch_size):
        yield corpus[i : i + batch_size]

In [ ]:
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

In [ ]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")

In [ ]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
#     pair=f"[CLS]:0 B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", cls_token_id),
        ("[SEP]", sep_token_id),
    ],
)


In [ ]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

In [ ]:
X = df['Keys'].tolist()
Y=  df['Status'].tolist()

In [ ]:
del df
gc.collect()

0

#### Building Model

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.01, random_state = 0)


In [ ]:
from transformers import DistilBertTokenizerFast
new_tokenizer = DistilBertTokenizerFast(tokenizer_object=tokenizer, model_max_length=512)

In [ ]:
train_encodings = new_tokenizer(X_train, truncation=True, padding=True)


In [ ]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))



In [ ]:
length = len(X_train)

In [ ]:
del  X_train , X_test, y_train, y_test,train_encodings
gc.collect()

0

In [ ]:
from transformers import TFDistilBertForSequenceClassification

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
optimizerr = tf.keras.optimizers.Adam(learning_rate=5e-2)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_19', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
#define the loss function
losss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#build the model
model.compile(optimizer=optimizerr,
              loss=losss,
              metrics=['accuracy'])
# train the model
model.fit(train_dataset.shuffle(length).batch(128),
          epochs=1,
          batch_size=128)

4641/4641 [==============================] - 5783s 1s/step - loss: 0.6956 - accuracy: 0.9822


In [ ]:
# # model evaluation on the test set
# model.evaluate(test_dataset.shuffle(len(X_test)).batch(256),
#                return_dict=True,
#                batch_size=256)

In [ ]:
del train_dataset
gc.collect()

15172

In [ ]:
import re

In [ ]:
def string_generator(text):
    string = ''
    p = re.compile('\w+')
    text = p.findall(text)
    for i in range(len(text)):
        if (re.fullmatch(re.compile(r"[A-Za-z]+"), text[i])):
            if (len(text[i])>1) :
                string+=" " + text[i]

    return string

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Copy of test.csv')

In [ ]:
%%time
data['Keys'] = data.apply(lambda x : string_generator(x[' Log']),axis =1 )

CPU times: user 34.4 s, sys: 170 ms, total: 34.6 s
Wall time: 34.6 s


In [ ]:
keys= data['Keys'].tolist()

In [ ]:
del data
gc.collect()

0

In [ ]:
data_encodings = new_tokenizer(keys, truncation=True, padding=True)

In [ ]:
import tensorflow as tf
data_dataset = tf.data.Dataset.from_tensor_slices((
    dict(data_encodings)
))

In [ ]:
del data_encodings
gc.collect()

0

In [ ]:
predict = model.predict(data_dataset)

595300/595300 [==============================] - 3184s 5ms/step


In [ ]:
del data_dataset
gc.collect()

0

In [ ]:
import numpy as np


In [ ]:
prediction = predict.logits
prediction

array([[-2.0941823,  2.094167 ],
       [-2.0941823,  2.094167 ],
       [-2.0941823,  2.094167 ],
       ...,
       [-2.0941823,  2.094167 ],
       [-2.0941823,  2.094167 ],
       [-2.0941823,  2.094167 ]], dtype=float32)

In [ ]:
pred = predict.logits
pred = pd.DataFrame(pred, columns=['Label1','Label2'])
pred

,Label1,Label2
0,-2.094182,2.094167
1,-2.094182,2.094167
2,-2.094182,2.094167
3,-2.094182,2.094167
4,-2.094182,2.094167
...,...,...
595295,-2.094182,2.094167
595296,-2.094182,2.094167
595297,-2.094182,2.094167
595298,-2.094182,2.094167


In [ ]:
final=[]

In [ ]:

for i in range(len(pred)):
  if pred['Label1'][i]>pred['Label2'][i]:
    final.append('abnormal')
  else:
    final.append('normal')

In [ ]:
final

['normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',

In [ ]:
final = pd.DataFrame(final, columns=[' Label'])

In [ ]:
final =final.reset_index()
final

,index,Label
0,0,normal
1,1,normal
2,2,normal
3,3,normal
4,4,normal
...,...,...
595295,595295,normal
595296,595296,normal
595297,595297,normal
595298,595298,normal


In [ ]:
final =final.rename(columns = {'index':'ID'})
final

,ID,Label
0,0,normal
1,1,normal
2,2,normal
3,3,normal
4,4,normal
...,...,...
595295,595295,normal
595296,595296,normal
595297,595297,normal
595298,595298,normal


In [ ]:
from google.colab import files
final.to_csv('final1.csv', encoding = 'utf-8-sig')
files.download('final1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
final

In [ ]:
final[' Label'].values